<a href="https://colab.research.google.com/github/JoaoPedroMoro/dio-transfer-learning/blob/main/dio_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

#if using Theano with GPU
#os.environ["KERAS_BACKEND"] = "tensorflow"

import random
import numpy as np
import keras

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Model

In [2]:
# Importando bibliotecas necessárias
import tensorflow as tf
import tensorflow_datasets as tfds

# Carregar o dataset cats_vs_dogs
(ds_train, ds_validation), ds_info = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:]'],
    with_info=True,
    as_supervised=True,
)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/23262 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/incomplete.HH0FGD_4.0.1/cats_vs_dogs-train.tfrecord*...:   0%…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.1. Subsequent calls will reuse this data.


In [3]:
# Função para redimensionar e normalizar as imagens
def preprocess(image, label):
    image = tf.image.resize(image, (224, 224))
    image = image / 255.0  # Normalizar para [0,1]
    return image, label

# Aplicar a função de preprocessamento ao dataset
batch_size = 32
ds_train = ds_train.map(preprocess).batch(batch_size).prefetch(tf.data.AUTOTUNE)
ds_validation = ds_validation.map(preprocess).batch(batch_size).prefetch(tf.data.AUTOTUNE)


In [4]:
# Importar o modelo pré-treinado
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')

# Congelar as camadas do modelo base
base_model.trainable = False

# Criar o modelo completo
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Uma saída para classificação binária
])


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [6]:
# Compilar o modelo
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Treinar o modelo
epochs = 5
history = model.fit(ds_train, validation_data=ds_validation, epochs=epochs)


Epoch 1/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 52s 78ms/step - accuracy: 0.9876 - loss: 0.0358 - val_accuracy: 0.9856 - val_loss: 0.0413
Epoch 2/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 42s 73ms/step - accuracy: 0.9895 - loss: 0.0329 - val_accuracy: 0.9858 - val_loss: 0.0417
Epoch 3/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 83s 75ms/step - accuracy: 0.9905 - loss: 0.0306 - val_accuracy: 0.9856 - val_loss: 0.0422
Epoch 4/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 42s 73ms/step - accuracy: 0.9910 - loss: 0.0287 - val_accuracy: 0.9858 - val_loss: 0.0428
Epoch 5/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 83s 75ms/step - accuracy: 0.9919 - loss: 0.0271 - val_accuracy: 0.9858 - val_loss: 0.0434


In [7]:
# Avaliar o modelo no conjunto de validação
loss, accuracy = model.evaluate(ds_validation)
print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")


146/146 ━━━━━━━━━━━━━━━━━━━━ 10s 67ms/step - accuracy: 0.9866 - loss: 0.0414
Loss: 0.0434274896979332
Accuracy: 0.9858125448226929
